In [1]:
import sys
import deepHSI

from pathlib import Path

import numpy as np
from lightning.pytorch import Trainer, seed_everything

# Custom module imports
from deepHSI.datamodule.components import HyperspectralDataset
from deepHSI.datamodule.components.utils import *

from deepHSI.datamodule.medical_datasets import BloodDetectionHSIDataModule

from deepHSI.datamodule.remote_sensing_datasets import *
# from deepHSI.datamodule.remote_sensing_datasets.paviaC import PaviaCDataModule

from deepHSI.models.architectures import HSIFCModel, SpectralSpatialCNN, HyperspectralCNNDetector
from deepHSI.models import HSIClassificationModule

seed_everything(42, workers=True)

# Importing from `lightning` instead of `pytorch_lightning`
import lightning as L

# PyTorch and metrics imports
import torch
from torchmetrics import F1Score, Precision, Recall

# from lightning import Trainer

torch.set_float32_matmul_precision("medium")

Seed set to 42


In [2]:
from pathlib import Path

# Specify the directory to save checkpoints
ckpt_dir = Path("/home/sayem/Desktop/deepHSI/notebooks/ckpt")

# Function to clear directory

def clear_directory(path: Path):
    if path.is_dir():
        for item in path.iterdir():
            if item.is_dir():
                clear_directory(item)
                item.rmdir()
            else:
                item.unlink()
    else:
        path.mkdir(parents=True, exist_ok=True)


# Clear and/or create the log and checkpoint directories
clear_directory(ckpt_dir)

In [3]:
# Define the parameters for the data module
data_dir = "/home/sayem/Desktop/deepHSI/data"  # Specify the directory where you want the data to be downloaded

# Include 'batch_size', 'num_workers', and 'num_classes' within the hyperparams dictionary
hyperparams = {
    "batch_size": 64,
    "num_workers": 24,
    "patch_size": 10,
    "center_pixel": True,
    "supervision": "full",
    "num_classes": 10,  # Define the number of classes in your dataset
}

# Assuming YourModel is defined elsewhere and num_classes is known
input_channels = 102

# Define custom metrics for the classification task using the updated hyperparams
custom_metrics = {
    "precision": Precision(
        num_classes=hyperparams["num_classes"], average="macro", task="multiclass"
    ),
    "recall": Recall(num_classes=hyperparams["num_classes"], average="macro", task="multiclass"),
    "f1": F1Score(num_classes=hyperparams["num_classes"], average="macro", task="multiclass"),
}

# model = HSIFCResNetModel(
#     input_channels=input_channels,
#     patch_size=hyperparams["patch_size"],  # Use patch_size from hyperparams
#     n_classes=hyperparams["num_classes"],  # Use num_classes from hyperparams
#     dropout=True,
# )

model = HyperspectralCNNDetector(
    input_channels=input_channels,
    patch_size=hyperparams["patch_size"],  # Use patch_size from hyperparams
    n_classes=hyperparams["num_classes"],  # Use num_classes from hyperparams
    dropout=True,
)

/home/sayem/anaconda3/envs/deepHSI/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv3d(


In [4]:
import torch

# Mapping for all optimizers available in PyTorch
optimizer_mapping = {
    'Adadelta': torch.optim.Adadelta,
    'Adagrad': torch.optim.Adagrad,
    'Adam': torch.optim.Adam,
    'AdamW': torch.optim.AdamW,
    'SparseAdam': torch.optim.SparseAdam,
    'Adamax': torch.optim.Adamax,
    'ASGD': torch.optim.ASGD,
    'LBFGS': torch.optim.LBFGS,
    'RMSprop': torch.optim.RMSprop,
    'Rprop': torch.optim.Rprop,
    'SGD': torch.optim.SGD,
}

# Mapping for all learning rate schedulers in PyTorch
scheduler_mapping = {
    'None': None,  # Option for no scheduler
    'LambdaLR': torch.optim.lr_scheduler.LambdaLR,
    'MultiplicativeLR': torch.optim.lr_scheduler.MultiplicativeLR,
    'StepLR': torch.optim.lr_scheduler.StepLR,
    'MultiStepLR': torch.optim.lr_scheduler.MultiStepLR,
    'ExponentialLR': torch.optim.lr_scheduler.ExponentialLR,
    'CosineAnnealingLR': torch.optim.lr_scheduler.CosineAnnealingLR,
    'ReduceLROnPlateau': torch.optim.lr_scheduler.ReduceLROnPlateau,
    'CyclicLR': torch.optim.lr_scheduler.CyclicLR,
    'OneCycleLR': torch.optim.lr_scheduler.OneCycleLR,
    'CosineAnnealingWarmRestarts': torch.optim.lr_scheduler.CosineAnnealingWarmRestarts,
}

In [5]:
# Parameters (tag this cell with 'parameters' in Jupyter)

# Default identifiers for optimizer and scheduler
optimizer_id = 'Adam' # Example default
scheduler_id = 'CosineAnnealingWarmRestarts'  # Default to not using a scheduler

In [6]:
torch_optimizer = optimizer_mapping[optimizer_id]
lr_scheduler = scheduler_mapping[scheduler_id]

In [7]:
import os
import wandb
from pytorch_lightning.loggers import WandbLogger

# Example model setup for demonstration
# model = ResNet50(...)  # Your model instantiation

# Dynamically get the model name
model_name = model.__class__.__name__

# Set the WandB notebook name for better organization
os.environ["WANDB_NOTEBOOK_NAME"] = "1.0-hsi-initial-data-exploration.ipynb"
wandb.login()

# Dynamically construct the run name to include key experiment details
dynamic_run_name = f"{optimizer_id}-with-{scheduler_id}-{model_name}"

# Extend the dynamic tags to include other important aspects of the experiment
dynamic_tags = [
    optimizer_id, scheduler_id, model_name,
    # Add more tags as needed, e.g., dataset name, special model configurations
]

print(dynamic_tags)

# Include additional notes for the experiment to describe unique aspects or goals
experiment_notes = f"Testing {optimizer_id} optimizer with \
    {scheduler_id} scheduler on {model_name}."

# Initialize WandbLogger with dynamic configurations
wandb_logger = WandbLogger(
    name=dynamic_run_name,
    project="PaviaC",
    save_dir="/home/sayem/Desktop/deepHSI/notebooks/wandb",
    offline=False,
    tags=dynamic_tags,
    notes=experiment_notes,  # Add the experiment notes
)

# Assume `hyperparams` is a dictionary containing your experiment-specific hyperparameters
# Update the WandB config with a comprehensive set of experiment details
wandb_logger.experiment.config.update({
    "optimizer": optimizer_id,
    "scheduler": scheduler_id,
    "model": model_name,
    # Include other experiment-specific hyperparameters
    **hyperparams,
})


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find 1.0-hsi-initial-data-exploration.ipynb.
wandb: Currently logged in as: k61. Use `wandb login --relogin` to force relogin
wandb: WARNING Path /home/sayem/Desktop/deepHSI/notebooks/wandb/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path /home/sayem/Desktop/deepHSI/notebooks/wandb/wandb/ wasn't writable, using system temp directory


['Adam', 'CosineAnnealingWarmRestarts', 'HyperspectralCNNDetector']


In [8]:
from functools import partial

import torch

# Hyperparameters from your YAML configuration
lr = 0.001  # learning rate
weight_decay = 0.001  # weight decay

# 'partial' creates a new function that when called will behave like 'torch.optim.Adam'
# with the given 'lr' and 'weight_decay' parameters pre-filled
optimizer = partial(torch_optimizer, lr=lr, weight_decay=weight_decay)

# Since 'optimizer_func' is a function created by 'partial', it won't be an instance of 'torch.optim.Optimizer'
# Therefore, the isinstance check is not applicable here. We can check if it's callable instead:
assert callable(optimizer)

# if lr_schedular == None:
#     scheduler = None
# else:
#     scheduler = partial(
#         lr_schedular,
#         mode="min",
#         factor=0.1,
#         patience=3,
#         threshold=0.0001,
#         threshold_mode="rel",
#         cooldown=0,
#         min_lr=0,
#         eps=1e-08,
#     )
#     Configure relevant parameters for CosineAnnealingWarmRestarts

scheduler = partial(
    lr_scheduler,
    T_0=10,  # Number of iterations for the first restart
    T_mult=2,  # A factor to increase T_i after a restart
    eta_min=0,  # Minimum learning rate
    last_epoch=-1  # The index of the last epoch
)
assert callable(scheduler)

# scheduler = None
# # # Since 'scheduler_func' is also a function created by 'partial', we check if it's callable:
# assert callable(scheduler)

In [9]:
from deepHSI.datamodule.transforms.hsi_transforms import *

# Instantiate individual transformations
normalize_transform = HSINormalize()
flip_transform = HSIFlip()
rotate_transform = HSIRotate()
noise_transform = HSISpectralNoise(mean=0.0, std=0.01)
shift_transform = HSISpectralShift(shift=2)
drop_transform = HSIRandomSpectralDrop(drop_prob=0.1)

# Compose transformations together
composed_transform = Compose([
    normalize_transform,
    flip_transform,
    rotate_transform,
    noise_transform,
    shift_transform,
    drop_transform
])

In [10]:
# Initialize the HSIClassificationModule with the model and other hyperparameters
hsi_classifier = HSIClassificationModule(
    net=model,  # Your model instance
    optimizer=optimizer,  # The Adam optimizer class from torch.optim
    scheduler=scheduler,  # The ReduceLROnPlateau scheduler class from torch.optim.lr_scheduler
    loss_fn=torch.nn.functional.cross_entropy,  # Using cross-entropy loss function
    num_classes=hyperparams["num_classes"],  # Number of classes from your hyperparameters
    custom_metrics=custom_metrics,  # Custom metrics dictionary if any
)

# # Initialize the PyTorch Lightning Trainer
# trainer = Trainer(max_epochs=10, precision='16-mixed', accelerator='gpu', devices=1)
max_epochs = 200

# Initialize the PaviaCDataModule with the updated arguments
pavia_c_datamodule = PaviaCDataModule(
    data_dir=data_dir, hyperparams=hyperparams, \
    transform=composed_transform,  # Pass hyperparams which now includes num_classes
)

In [11]:
# Callbacks
# Define the EarlyStopping callback
early_stop_callback = L.pytorch.callbacks.EarlyStopping(
    monitor="val/f1",  # Specify the metric to monitor
    patience=20,  # Number of epochs with no improvement after which training will be stopped
    verbose=True,  # Whether to print logs to stdout
    mode="max",  # In 'min' mode, training will stop when the quantity monitored has stopped decreasing
    check_on_train_epoch_end=False,
)

# from lightning.pytorch.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback
model_checkpoint = L.pytorch.callbacks.ModelCheckpoint(
    monitor="val/f1",  # Metric to monitor
    dirpath=str(ckpt_dir),  # Convert Path object to string, Directory to save checkpoints
    filename="best-checkpoint-{epoch:02d}-{val/f1:.2f}",  # Checkpoint file name
    save_top_k=1,  # Save only the best checkpoint
    mode="max",  # 'max' because we want to maximize 'val/f1'
    verbose=True,  # Print a message when a new best is found
    auto_insert_metric_name=False,  # Prevents metric names being inserted into filename automatically
)

rich_pbar_callback = L.pytorch.callbacks.RichProgressBar(
    refresh_rate=1,
    leave=True,
)

lr_monitor_callback = L.pytorch.callbacks.LearningRateMonitor(logging_interval='epoch') 

In [12]:
from deepHSI.utils.custom_callbacks import (
    ConfusionMatrixLoggerCallBack, InitialBatchSizeFinder,
)

confusion_matrix_callback = ConfusionMatrixLoggerCallBack()

from deepHSI.utils.custom_callbacks import (
    DynamicLRFinder,
    InitialLRFinder,
)

# from deepHSI.utils.custom_callbacks import InitialBatchSizeFinder
# # Instantiate the DynamicLRFinder callback with the defined milestones
# dynamic_lr_finder = InitialLRFinder(min_lr=1e-8, max_lr=1, \
#     num_training_steps=100, mode='exponential')

# Instantiate the DynamicLRFinder callback with the defined milestones
lr_finder_callback = InitialLRFinder()
batch_finder_callback = InitialBatchSizeFinder()

In [13]:
# Initialize the PyTorch Lightning Trainer with fast_dev_run enabled
trainer = L.Trainer(
    fast_dev_run=False,  # Enable fast_dev_run
    precision="16-mixed",  # Use 16-bit precision
    accelerator="auto",  # Specify the accelerator as GPU
    max_epochs=max_epochs,
    log_every_n_steps=3,
    callbacks=[
        # lr_finder_callback,
        early_stop_callback,
        model_checkpoint,
        confusion_matrix_callback,
        # batch_finder_callback,
        lr_monitor_callback,
    ],  # rich_pbar_callback],
    logger=wandb_logger,
    deterministic=False,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(hsi_classifier, datamodule=pavia_c_datamodule)

Dataset 'PaviaC' already exists. Skipping download.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                     | Params
-------------------------------------------------------
0 | net       | HyperspectralCNNDetector | 215 K 
1 | precision | MulticlassPrecision      | 0     
2 | recall    | MulticlassRecall         | 0     
3 | f1        | MulticlassF1Score        | 0     
-------------------------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.863     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 25, global step 217230: 'val/f1' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 26, global step 225585: 'val/f1' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val/f1 did not improve in the last 20 records. Best score: 0.613. Signaling Trainer to stop.
Epoch 27, global step 233940: 'val/f1' was not in top 1


In [15]:
# Fit the model using the train dataset from the data module
dictionary = trainer.test(hsi_classifier, pavia_c_datamodule, verbose=True)
# trainer.fit(hsi_module, datamodule=pavia_c_datamodule)tree
# Use train_dataloader() instead of train_dataset

Dataset 'PaviaC' already exists. Skipping download.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

test_preds_np shape: (114576,), first few elements: [0 0 0 0 0]
test_targets_np shape: (114576,), first few elements: [0 0 0 0 0]


TypeError: Expected a list as "images", found <class '_io.BytesIO'>

In [ ]:
# After training or experiments are done, finish the WandB run
wandb.finish()